In [4]:
# importing the required packages and libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import sqlite3
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer as sno
setofstopwords=set(stopwords.words('english'))
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# connecting to the sqlite database
con= sqlite3.connect(r'C:\Users\harit\Downloads\AmazonFoodReviews\Data\database.sqlite')
# delete the reviews with th score 3 as it is average
data= pd.read_sql_query('''
SELECT * FROM Reviews WHERE Score!=3
''',con)
data.shape

(525814, 10)

In [6]:
# classify the score values based on the score > 3 or not as positive and negative
data['Score']=data['Score'].map(lambda x:'Positive' if x>3 else 'Negative')
# sort the data based on product ID
sorteddata= data.sort_values('ProductId',axis=0)
# dropping the duplicates
finaldata= sorteddata.drop_duplicates(subset={'UserId','ProfileName',\
        'Time','Text'}, keep='first',inplace=False)
finaldata= finaldata[finaldata['HelpfulnessNumerator'] <= finaldata['HelpfulnessDenominator']]
data= finaldata.sort_values('Time',axis=0)
data.shape

(364171, 10)

In [7]:
def cleanhtml(sentance): 
    cleanedData= re.sub(re.compile('<.*?>'),' ',sentance)
    return cleanedData
#function for removing punctuations chars
def cleanpunc(sent):
    cleanedData= re.sub(r'[?|!|\'|"|#]',r'',sent)
    cleanedData= re.sub(r'[.|,|)|(|\|/]',r'',sent)
    return cleanedData
snowstemmer= sno('english')

i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
for sent in data['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                s=(snowstemmer.stem(cleaned_words.lower())).encode('utf8')
                filtered_sentence.append(s)
                if(data['Score'].values)[i] =='Positive':
                    all_positive_words.append(s)
                if(data['Score'].values)[i] =='Negative':
                    all_negative_words.append(s)
            else:
                continue
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    final_string.append(str1)

# storing data till now
data['CleanedText']=final_string 
#adding a column of CleanedText which displays the data after pre-processing of the review 
data['CleanedText']=data['CleanedText'].str.decode("utf-8")
    # store final table into an SQlLite table for future.
con = sqlite3.connect('cleanedTextData.sqlite')
c=con.cursor()
con.text_factory = str
data.to_sql('Reviews', con,  schema=None, if_exists='replace', \
        index=True, index_label=None, chunksize=None, dtype=None)
con.close()

In [8]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,this witti littl book make son laugh loud reci...
138683,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,Positive,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,can rememb see the show when air televis year ...
417839,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,Positive,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,beetlejuic well written movi everyth about fro...
346055,374359,B00004CI84,A344SMIA5JECGM,Vincent P. Ross,1,2,Positive,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",twist rumplestiskin captur film star michael k...
417838,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,Positive,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,beetlejuic excel and funni movi keaton hilari ...


In [9]:
data['Text'][2]

'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.'

In [10]:
data['CleanedText'][2]

'this confect that has been around few centuri light pillowi citrus gelatin with nut this case filbert and cut into tini squar and then liber coat with powder sugar and tini mouth heaven not too chewi and veri flavor high recommend this yummi treat you are familiar with the stori lion the witch and the this the treat that seduc edmund into sell out his brother and sister the witch'

In [12]:
# storing the intermediate data in pickle files
import pickle

data.to_pickle("datafile.pkl")

In [13]:
#extracting some partof the data with the textand score and excluding the other columns
Data= data[:100000]
Data= Data[['CleanedText','Score']]
Data['Score']= Data['Score'].map(lambda x:1 if x=='Positive' else 0)
Data_x= Data['CleanedText']
Data_y= Data['Score']

In [14]:
Data_x.index= [i for i in range(0,10**5)]
Data_x

0        this witti littl book make son laugh loud reci...
1        can rememb see the show when air televis year ...
2        beetlejuic well written movi everyth about fro...
3        twist rumplestiskin captur film star michael k...
4        beetlejuic excel and funni movi keaton hilari ...
                               ...                        
99995    this veri tasti protein shake and give you nic...
99996    this the best tast oliv and healthi for you to...
99997    sack melitta coffe label fine grind blanc noir...
99998    final babi food that tast love that great prot...
99999    first ventur salt and happi bought this and sa...
Name: CleanedText, Length: 100000, dtype: object

In [15]:
# collecting all words in single list
list_= []
for i in Data_x:
    list_ += i
list_= ''.join(list_)
allWords=list_.split()

In [16]:
vocab= set(allWords)

In [17]:
vocabulary_list= list(vocab)

In [18]:
#frequency dictionary
freq_dict= {}
for word in vocabulary_list:
    freq_dict[word]= allWords.count(word)

In [19]:
freq_dict

{'blehthis': 1,
 'organall': 1,
 'bettermani': 1,
 'themomg': 1,
 'dusk': 1,
 'shorteningbut': 1,
 'helpbeen': 1,
 'tilth': 2,
 'supermartpurchas': 1,
 'flavornice': 2,
 'wafflveri': 1,
 'vanillacolumbian': 1,
 'manythi': 1,
 'preservthis': 4,
 'foundgiven': 1,
 'adhespoor': 1,
 'teethreceiv': 1,
 'rightway': 1,
 'ourfar': 1,
 'gumballsw': 1,
 'berrri': 3,
 'lunchgreat': 3,
 'stanolssterol': 1,
 'sugarabsolut': 1,
 'crockeri': 9,
 'chucklforget': 1,
 'likeoften': 1,
 'dervish': 1,
 'wasmin': 1,
 'strongeasi': 1,
 'extol': 5,
 'whiski': 12,
 'stillalon': 1,
 'invoic': 16,
 'committ': 3,
 'wrongsplenda': 1,
 'breakfastour': 1,
 'nugent': 1,
 'spinach': 201,
 'readithis': 4,
 'boar': 2,
 'burmathis': 1,
 'againwhatev': 1,
 'comereceiv': 1,
 'rainbow': 19,
 'amazonwell': 3,
 'themonli': 1,
 'betterso': 1,
 'madwoman': 2,
 'bracer': 1,
 'christmalizano': 1,
 'mealbut': 1,
 'oounc': 1,
 'wheatitem': 1,
 'icethis': 11,
 'davisonnew': 1,
 'panthese': 1,
 'variationson': 1,
 'largthis': 3,
 'fr

In [20]:
import pickle
with open('freq_dict.pkl','wb') as file:
    pickle.dump(freq_dict,file)

In [21]:
from operator import itemgetter
sorted_list= []
for k, v in sorted(freq_dict.items(), key=itemgetter(1),reverse=True):
    sorted_list.append(k)

In [22]:
sorted_list

['the',
 'and',
 'this',
 'for',
 'that',
 'with',
 'you',
 'have',
 'but',
 'are',
 'not',
 'they',
 'was',
 'like',
 'tast',
 'flavor',
 'them',
 'these',
 'good',
 'tea',
 'one',
 'use',
 'can',
 'product',
 'veri',
 'great',
 'just',
 'tri',
 'all',
 'from',
 'love',
 'make',
 'has',
 'when',
 'get',
 'more',
 'other',
 'will',
 'than',
 'coffe',
 'had',
 'out',
 'would',
 'some',
 'buy',
 'food',
 'onli',
 'eat',
 'about',
 'time',
 'your',
 'find',
 'realli',
 'also',
 'best',
 'much',
 'too',
 'littl',
 'order',
 'even',
 'amazon',
 'becaus',
 'drink',
 'which',
 'were',
 'price',
 'bag',
 'there',
 'store',
 'been',
 'mix',
 'what',
 'chocol',
 'ani',
 'better',
 'well',
 'box',
 'sugar',
 'now',
 'year',
 'their',
 'after',
 'sweet',
 'found',
 'day',
 'dog',
 'want',
 'then',
 'high',
 'look',
 'our',
 'give',
 'cup',
 'over',
 'first',
 'add',
 'water',
 'brand',
 'recommend',
 'most',
 'she',
 'made',
 'think',
 'packag',
 'way',
 'who',
 'treat',
 'two',
 'nice',
 'work',


In [23]:
Data_x[1]

'can rememb see the show when air televis year ago when was child sister later bought the which have this day thirti somethingi use this seri book song when did student teach for preschool turn the whole school now purchas along with the book for children the tradit live'

In [29]:
top_words= 5000
sorted_list= sorted_list[:5000]
import pickle
file_name = "sortedlist.pkl"

open_file = open(file_name, "wb")
pickle.dump(sorted_list, open_file)
open_file.close()

In [26]:
column=[]
for sent in Data_x:
    lis=[]
    for word in sent.split():
        if word in sorted_list:
            lis.append(word)
    column.append(' '.join(lis))

In [27]:
with open('column.pkl','wb') as file:
    pickle.dump(column,file)

In [28]:
final_x=[]
for sent in Data_x:
    lis=[]
    for word in sent.split():
        if word in sorted_list:
            lis.append(sorted_list.index(word)+1)
    final_x.append(lis)

In [29]:
Xtest= final_x[:30000]
Ytest= Data_y[:30000]
Xtrain= final_x[30000:]
Ytrain= Data_y[30000:]

In [30]:
print(Xtrain[1])

[8, 216, 209, 1977, 106, 8, 368, 25, 3968, 527, 3224, 3224, 721, 17, 488, 1100, 2, 583, 127, 5, 1069, 474, 5, 11, 698, 8, 47, 28, 1, 557, 16, 3224, 3224, 9, 322, 1, 31, 89, 99, 12, 10, 156, 621, 42, 91, 151, 69, 541, 35, 1133, 17, 61, 5, 8, 352, 1, 69]


In [31]:
from keras.preprocessing import sequence
max_review_length=600
Xtrain = sequence.pad_sequences(Xtrain, maxlen=max_review_length)
Xtest= sequence.pad_sequences(Xtest, maxlen=max_review_length)

In [32]:
print(Xtrain[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [33]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5642568739446880652
xla_global_id: -1
]


In [34]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

In [35]:
# create the model
embedding_vecor_length = 32
# building the sequential model
model = Sequential()
# adding the enbedded layer
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
# adding the LSTM
model.add(LSTM(100))
# activation function is choosen as sigmoid as we havetwo different output values
model.add(Dense(1, activation='sigmoid'))
# configuring the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 600, 32)           160032    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,333
Trainable params: 213,333
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
history=model.fit(Xtrain, Ytrain, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(Xtest, Ytest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
1094/1094 [==============================] - 630s 574ms/step - loss: 0.2757 - accuracy: 0.8948
Epoch 2/10
1094/1094 [==============================] - 706s 645ms/step - loss: 0.1923 - accuracy: 0.9240
Epoch 3/10
1094/1094 [==============================] - 781s 713ms/step - loss: 0.1783 - accuracy: 0.9308
Epoch 4/10
1094/1094 [==============================] - 757s 692ms/step - loss: 0.1554 - accuracy: 0.9398
Epoch 5/10
1094/1094 [==============================] - 716s 654ms/step - loss: 0.1423 - accuracy: 0.9444
Epoch 6/10
1094/1094 [==============================] - 716s 654ms/step - loss: 0.1304 - accuracy: 0.9497
Epoch 7/10
1094/1094 [==============================] - 747s 683ms/step - loss: 0.1200 - accuracy: 0.9539
Epoch 8/10
1094/1094 [==============================] - 608s 556ms/step - loss: 0.1053 - accuracy: 0.9605
Epoch 9/10
1094/1094 [==============================] - 727s 664ms/step - loss: 0.0921 - accuracy: 0.9664
Epoch 10/10
1094/1094 [=======================

In [37]:
# Save the model
from keras.models import load_model
model.save('model.h5')
# saving the model
model1 = load_model('model.h5')

In [38]:
# create the model with adding dropout 
embedding_vecor_length = 32
model2 = Sequential()
model2.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model2.add(LSTM(100,return_sequences=True))
model2.add(Dropout(0.25))
model2.add(LSTM(80))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 600, 32)           160032    
                                                                 
 lstm_1 (LSTM)               (None, 600, 100)          53200     
                                                                 
 dropout (Dropout)           (None, 600, 100)          0         
                                                                 
 lstm_2 (LSTM)               (None, 80)                57920     
                                                                 
 dropout_1 (Dropout)         (None, 80)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 81        
                                                                 
Total params: 271,233
Trainable params: 271,233
Non-tr

In [39]:
history2= model2.fit(Xtrain, Ytrain, epochs=10, batch_size=64)# Final evaluation of the model
scores = model2.evaluate(Xtest, Ytest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
1094/1094 [==============================] - 1521s 1s/step - loss: 0.2398 - accuracy: 0.9095
Epoch 2/10
1094/1094 [==============================] - 1636s 1s/step - loss: 0.1793 - accuracy: 0.9325
Epoch 3/10
1094/1094 [==============================] - 1689s 2s/step - loss: 0.1536 - accuracy: 0.9425
Epoch 4/10
1094/1094 [==============================] - 1700s 2s/step - loss: 0.1359 - accuracy: 0.9495
Epoch 5/10
1094/1094 [==============================] - 1774s 2s/step - loss: 0.1184 - accuracy: 0.9575
Epoch 6/10
1094/1094 [==============================] - 1739s 2s/step - loss: 0.1027 - accuracy: 0.9631
Epoch 7/10
1094/1094 [==============================] - 1713s 2s/step - loss: 0.0980 - accuracy: 0.9639
Epoch 8/10
1094/1094 [==============================] - 1608s 1s/step - loss: 0.0840 - accuracy: 0.9706
Epoch 9/10
1094/1094 [==============================] - 1622s 1s/step - loss: 0.0695 - accuracy: 0.9758
Epoch 10/10
1094/1094 [==============================] - 1634s 1

In [40]:
# Save the model
model2.save('model2.h5')
model3 = load_model('model2.h5')

In [ ]:
# with open('model.pkl','wb') as file:
#     pickle.dump(model,file)
# with open('model2.pkl','wb') as file:
#     pickle.dump(model2,file)

In [41]:
# making a function which convert sentance to requred vectorized format that will feed well in model

def cleanhtml(sentance): #substitute expression contained in <> with ' '
    cleaned= re.sub(re.compile('<.*?>'),' ',sentance)
    return cleaned
#function for removing punctuations chars
def cleanpunc(sentance):
    cleaned= re.sub(r'[?|!|\'|"|#]',r'',sentance)
    cleaned= re.sub(r'[.|,|)|(|\|/]',r'',sentance)
    return cleaned
snowstemmer= sno('english')

def predict_this(sentance):
    i=0
    str1=' '
    final_string=[]
    all_positive_words=[] # store words from +ve reviews here
    all_negative_words=[] # store words from -ve reviews here.
    sent= sentance
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        # we have used cleanpunc(w).split(), one more split function here 
        # because consider w="abc.def", cleanpunc(w) will return "abc def"
        # if we dont use .split() function then we will be considring "abc def" 
        # as a single word, but if you use .split() function we will get "abc", "def"
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                s=(snowstemmer.stem(cleaned_words.lower())).encode('utf8')
                filtered_sentence.append(s)
                if(data['Score'].values)[i] =='Positive':
                    all_positive_words.append(s)
                if(data['Score'].values)[i] =='Negative':
                    all_negative_words.append(s)
            else:
                continue
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    final_string.append(str1)

    final_string
    for i in final_string:
        final_string=i.decode("utf-8")

    lis=[]
    for word in final_string.split():
        if word in sorted_list:
            lis.append(sorted_list.index(word)+1)

    final_string= lis
    final_string = sequence.pad_sequences([final_string], maxlen=max_review_length)
    print(final_string)
    what= ''
    if (round(float(model3.predict(final_string)))==1):
        what= 'Positive'
        acc= round(float(model3.predict(final_string))*100,2)
    else:
        what='Negative'
        acc= 100- round(float(model3.predict(final_string))*100,2)
    print(what,'review with',acc,'% Accuracy')

In [42]:
sentance= 'food was very bad in taste'
predict_this(sentance)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 

In [43]:
sentance= 'food was medium tasty'
predict_this(sentance)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 